In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset .csv')

df = df[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Votes','City', 'Address']]
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df['Cuisines'] = df['Cuisines'].fillna("Unknown")

#vectorize cuisines using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Cuisines'])

scaler = MinMaxScaler()
numerical_features = scaler.fit_transform(df[['Price range', 'Aggregate rating']])

#combine all features
combined_features = hstack([tfidf_matrix, numerical_features])

#define the recommendation function
def recommend_restaurants(cuisine_preference, price_range, top_n=10):
    #filter restaurants by price range
    filtered_df = df[df['Price range'] == price_range].reset_index(drop=True)

    if filtered_df.empty:
        return "No restaurants found for this price range."

    #TF-IDF for filtered data
    tfidf_filtered = TfidfVectorizer(stop_words='english')
    tfidf_matrix_filtered = tfidf_filtered.fit_transform(filtered_df['Cuisines'])

    #normalize the numerical features for filtered data
    numerical_filtered = scaler.fit_transform(filtered_df[['Price range', 'Aggregate rating']])

    #combine features again
    combined_filtered = hstack([tfidf_matrix_filtered, numerical_filtered])

    #transform the input cuisine
    user_input_vec = tfidf_filtered.transform([cuisine_preference])

    #combine with user price range and dummy rating (e.g., 0.0)
    user_price_df = pd.DataFrame([[price_range, 0.0]], columns=['Price range', 'Aggregate rating'])
    user_price = scaler.transform(user_price_df)
    user_features = hstack([user_input_vec, user_price])

    similarity = cosine_similarity(user_features, combined_filtered).flatten()

    #top similar restaurant
    top_indices = similarity.argsort()[::-1][:top_n]
    recommendations = filtered_df.iloc[top_indices].sort_values(by='Aggregate rating', ascending=False)

    return recommendations[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'Votes', 'City', 'Address']]

if __name__ == "__main__":
    user_cuisine = input("Enter your preferred cuisine (e.g., Chinese, Pizza, North Indian): ")
    user_price_range = int(input("Enter your preferred price range (1 to 4): "))

    result = recommend_restaurants(user_cuisine, user_price_range, top_n=10)

pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.width', None)

print(f"\nTop 10 restaurant recommendations based on your preferences:\n")
for index, row in result.iterrows():
    print(f"Restaurant Name : {row['Restaurant Name']}")
    print(f"Cuisines        : {row['Cuisines']}")
    print(f"Price Range     : {row['Price range']}")
    print(f"Rating          : {row['Aggregate rating']}")
    print(f"Votes           : {row['Votes']}")
    print(f"City            : {row.get('City', 'N/A')}")
    print(f"Address         : {row['Address']}")
    print("-" * 80)



Enter your preferred cuisine (e.g., Chinese, Pizza, North Indian): chiness
Enter your preferred price range (1 to 4): 1

Top 10 restaurant recommendations based on your preferences:

Restaurant Name : Ekim Burgers
Cuisines        : Fast Food
Price Range     : 1
Rating          : 4.5
Votes           : 195
City            : Wellington City
Address         : 257 Cuba Street, Te Aro, Wellington City
--------------------------------------------------------------------------------
Restaurant Name : The Royal Peasant
Cuisines        : Bar Food
Price Range     : 1
Rating          : 4.4
Votes           : 546
City            : Athens
Address         : 1675 S. Lumpkin St., Athens, GA 30606
--------------------------------------------------------------------------------
Restaurant Name : Grit
Cuisines        : International, Southern, Vegetarian
Price Range     : 1
Rating          : 4.2
Votes           : 800
City            : Athens
Address         : 199 Prince Ave, Athens, GA 30601
--------------